<a href="https://colab.research.google.com/github/Saadkhalid913/ML-Practice/blob/main/EmotionLinesModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## importing standard modules for NLP

In [106]:
import numpy as np
import pandas as pd 
import re
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.corpus import stopwords
from nltk.stem import wordnet, WordNetLemmatizer 
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Getting data from CSV and separating features & labels

In [107]:
data = pd.read_csv("output.csv", header=None, delimiter="~").values
x = data[ : , : -1]
y = data[ : , -1 : ]

print(x.shape)
print(y.shape)

(10558, 1)
(10558, 1)


Cleaning function for data

In [108]:
EnglishStopwords = set(stopwords.words("english"))

def CleanData(sentences):
  lemma = WordNetLemmatizer()
  '''
    Takes a 2D numpy array of sentences of 
    shape (n, 1) where n is the number of 
    sentences. Returns sentences without 
    stopwords, non-alphanumeric characters, 
    and performs lemmatization
  '''
  cleaned = []

  for sentence in sentences:
    sentence = sentence[0]
    sentence = re.sub("^a-zA-Z", " ", sentence)
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = sentence.split()
    sentence = [lemma.lemmatize(word) for word in sentence if word not in EnglishStopwords]
    sentence = " ".join(sentence)
    cleaned.append(sentence)

  return cleaned 

x = CleanData(x) # cleaning features 

## applying tokenization to features 

In [109]:
def FitAndTokenize(sentences):
  tokenizer = tf.keras.preprocessing.text.Tokenizer()
  tokenizer.fit_on_texts(sentences)
  result = tokenizer.texts_to_sequences(sentences)
  result = tf.keras.preprocessing.sequence.pad_sequences(result, maxlen=90)
  return result, tokenizer 

def Tokenize(sentences, tokenizer):
  result = tokenizer.texts_to_sequences(sentences)
  result = tf.keras.preprocessing.sequence.pad_sequences(result, maxlen=90)
  return result 

x, tokenizer = FitAndTokenize(x)

In [110]:
num_words = len(tokenizer.index_word) + 1

Encoding labels using one hot vector encoding 

In [111]:
encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()
KEY = encoder.categories_[0]


In [118]:
def CreateModel():
  '''
    creating tf.keras model 
  '''

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Embedding(num_words, 240, input_length=90))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=240, activation="relu"))
  model.add(tf.keras.layers.Dense(units=256, activation="relu"))
  model.add(tf.keras.layers.Dense(units=128, activation="relu"))
  model.add(tf.keras.layers.Dense(units=32, activation="relu"))
  model.add(tf.keras.layers.Dense(units=8, activation="softmax"))
  model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
  return model 


In [ ]:
ann = CreateModel()
ann.summary()

In [120]:
print(x.shape)
print(y.shape)

(10558, 90)
(10558, 8)


In [121]:
ann.fit(x, y, epochs=25, batch_size=100)

Epoch 1/25
106/106 [==============================] - 11s 96ms/step - loss: 1.6077 - accuracy: 0.4418
Epoch 2/25
106/106 [==============================] - 10s 94ms/step - loss: 1.4317 - accuracy: 0.5027
Epoch 3/25
106/106 [==============================] - 10s 96ms/step - loss: 1.0989 - accuracy: 0.6235
Epoch 4/25
106/106 [==============================] - 10s 95ms/step - loss: 0.8045 - accuracy: 0.7247
Epoch 5/25
106/106 [==============================] - 10s 95ms/step - loss: 0.6339 - accuracy: 0.7855
Epoch 6/25
106/106 [==============================] - 10s 95ms/step - loss: 0.5299 - accuracy: 0.8219
Epoch 7/25
106/106 [==============================] - 10s 98ms/step - loss: 0.4604 - accuracy: 0.8378
Epoch 8/25
106/106 [==============================] - 10s 97ms/step - loss: 0.4196 - accuracy: 0.8536
Epoch 9/25
106/106 [==============================] - 10s 98ms/step - loss: 0.3920 - accuracy: 0.8633
Epoch 10/25
106/106 [==============================] - 10s 98ms/step - loss: 0.363